In [ ]:
import random
import numpy as np
import time
from numpy.linalg import solve
import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import sys
import platform
import psutil
import subprocess


print("CPU: Intel(R) Xeon(R) CPU @ 2.20GHz\n"
      f"RAM: 12.68 GB\n"
      f"Versão do compilador: {sys.version}\n"
      f"Sistema operacional: {sys.platform}")

CPU: Intel(R) Xeon(R) CPU @ 2.20GHz
RAM: 12.68 GB
Versão do compilador: 3.7.14 (default, Sep  8 2022, 00:06:44) 
[GCC 7.5.0]
Sistema operacional: linux


#**Método de Gauss**

In [ ]:
def gauss(a,b):
  n = len(b) #n is matrix size
  it = 0
  #Elimination phase
  for k in range(0,n-1): #k is matrix row
    for i in range(k+1,n): #i is matrix col
      it += 1
      if a[i,k] != 0.0:
        if a[i,k]/a[k,k] != float('inf') and a[i,k]/a[k,k] != float('-inf'):
          factor = a[i,k]/a[k,k]
          a[i,k+1:n] = a[i,k+1:n] - np.multiply(factor,a[k,k+1:n])
          b[i] = b[i] - np.multiply(factor,b[k])

  #Back substitution
  for k in range(n-1,-1,-1):
    it += 1
    if (b[k] - np.dot(a[k,k+1:n],b[k+1:n]))/a[k,k] != float('inf') and (b[k] - np.dot(a[k,k+1:n],b[k+1:n]))/a[k,k] != float('-inf'):
      b[k] = (b[k] - np.dot(a[k,k+1:n],b[k+1:n]))/a[k,k]

  return b, it

#**Método LU**

In [ ]:
def magalu(matrix, b):
  n = len(matrix)

  it = 0
  p = []
  for i in range(0, n):
    it += 1
    p.append(i)
  
  
  for k in range(0, n-1):
    pv = abs(matrix[k][k])
    r = k
    for i in range(k+1, n):
      it += 1
      if abs(matrix[i][k]) > pv:
        pv = abs(matrix[i][k])
        r = i

    if pv == 0:
      print("A matrix é singular!")
      return
    if r != k:
      aux = p[k]
      p[k] = p[r]
      p[r] = aux
      for j in range(0, n):
        aux = matrix[k][j]
        matrix[k][j] = matrix[r][j]
        matrix[r][j] = aux

    for i in range(k+1, n):
      m = matrix[i][k]/matrix[k][k]
      matrix[i][k] = m
      for j in range(k+1, n):
        it += 1
        matrix[i][j] -= m*matrix[k][j]


  c = n * [0]
  for i in range(0, n):
    it += 1
    r = p[i]
    c[i] = b[r]

  y = n * [0]
  for i in range(0, n):
    soma = 0
    for j in range(0, i):
      it += 1
      soma += matrix[i][j]*y[j]
    y[i] = (c[i] - soma)

  x = n * [0]
  for i in range(n-1, -1, -1):
    soma = 0
    for j in range(i+1, n):
      it += 1
      soma += matrix[i][j]*x[j]
    x[i] = (y[i] - soma)/matrix[i][i]

  x = np.array(x)
  return x, it

#**Método de Gauss-Jacobi**

In [ ]:
def jacobi(A,b,n):
  x = len(A) * [0]
  D = np.diag(A)
  R = A-np.diagflat(D)
  it = 0
  for i in range(n):
    x = (b-np.dot(R,x))/D
  
  x = np.array(x)
  return x, n

#**Método de Gauss-Seidel**

In [ ]:
def update_solution(
    coefficient_matrix, solution_vector, constant_vector
):
    """
    Function to update the solution
    :param coefficient_matrix: list, Coefficient Matrix
    :param solution_vector: list, Solution Vector
    :param constant_vector: list, Constant Vector
    :return: list, Updated Solution Vector
    Example:
a = [[10, -1, 2, 0], [-1, 11, -1, 3], [2, -1, 10, -1], [0, 3, -1, 8]]
    >>> x = [0, 0, 0, 0]
    >>> b = [6, 25, -11, 15]
    >>> update_solution(a, x, b)
    [0.6, 2.3272727272727276, -0.9872727272727271, 0.8788636363636363]
    """
    it = 0
    n = len(coefficient_matrix)  # number of variables
    for i in range(0, n):
        # Update each variable
        d = constant_vector[i]
        for j in range(0, n):
            it += 1
            if i != j:
                d -= coefficient_matrix[i][j] * solution_vector[j]
        solution_vector[i] = d / coefficient_matrix[i][i]
    return solution_vector

def gauss_seidel(
    coefficient_matrix,
    constant_vector,
    maximum_iterations,
):
    solution_vector = len(coefficient_matrix) * [0]
    
    """
    Function to perform Gauss Seidel Iteration
    :param coefficient_matrix: list, Coefficient Matrix
    :param solution_vector: list, Solution Vector
    :param constant_vector: list, Constant Vector
    :param maximum_iterations: int, Maximum Iterations
    :return: None
    Example:
a = [[10, -1, 2, 0], [-1, 11, -1, 3], [2, -1, 10, -1], [0, 3, -1, 8]]
    >>> x = [0, 0, 0, 0]
    >>> b = [6, 25, -11, 15]
    >>> gauss_seidel(a, x, b, 100)
    [1.0, 2.0, -1.0, 1.0]
    """

    for i in range(0, maximum_iterations):
        solution_vector = update_solution(
            coefficient_matrix, solution_vector, constant_vector
        )
        # print("Iteration {}: {}".format(i, x))

    solution_vector = np.array(solution_vector)
    return solution_vector, maximum_iterations

#**Funções para criar matrizes densas e matrizes esparsas**

In [ ]:
def criarMatrizDensa(ordem):
  A = np.random.randint(-10, ordem+100, size=(ordem, ordem), dtype=float)
  b = np.random.randint(-10, ordem+100, size=(1, ordem), dtype=float)
  for i in range(ordem):
      if np.sum(A[i]) > A[i][i]:
        A[i][i] = np.sum(A[i])*2
  return A, b[0]

def criarMatrizEsparsa(ordem):
  A = np.random.randint(-10, ordem+100, size=(ordem, ordem))
  b = np.random.randint(-10, ordem+1, size=(1, ordem))
  for i in range(ordem):
    for j in range(ordem):
      chance = random.randint(0, 4)
      if chance > 0 and i != j:
        A[i][j] = 0
      else:
        A[i][j] = random.randint(-10, ordem+100)
        
  return A, b[0]

#**Gráfico dos sistemas densos**

In [ ]:
n = [10]

for i in range(20, 301, 10):
  n.append(i)

timeGauss = []
timeLu = []
timeJacobi = []
timeSeidel = []

itGauss = []
itLu = []
itJacobi = []
itSeidel = []

for item in n:
  A0, b = criarMatrizDensa(item)
  A1 = A0.copy()
  A2 = A0.copy()
  A3 = A0.copy()

  timeStart = time.time()
  G = gauss(A0,b)
  timeEnd = time.time()
  timeGauss.append((timeEnd - timeStart)*1000)
  itGauss.append(G[1])

  timeStart = time.time()
  M = magalu(A1,b)
  timeEnd = time.time()
  timeLu.append((timeEnd - timeStart)*1000)
  itLu.append(M[1])

  timeStart = time.time()
  J = jacobi(A2,b, 25)
  timeEnd = time.time()
  timeJacobi.append((timeEnd - timeStart)*1000)
  itJacobi.append(J[1])

  timeStart = time.time()
  S = gauss_seidel(A3,b, 25)
  timeEnd = time.time()
  timeSeidel.append((timeEnd - timeStart)*1000)
  itSeidel.append(S[1])

print(timeGauss)
print(timeLu)
print(timeJacobi)
print(timeSeidel)
print(itGauss)
print(itLu)
print(itJacobi)
print(itSeidel)

timeGraph = go.Figure()
timeGraph.update_layout(
    title="Gráfico do Tempo de Execução - Sistemas Densos",
    xaxis_title="Ordem da Matriz",
    yaxis_title="Tempo em ms",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)

timeGraph.add_trace(go.Scatter(x=n, y=timeGauss,mode='lines',name='Gauss'))
timeGraph.add_trace(go.Scatter(x=n, y=timeLu,mode='lines',name='Lu'))
timeGraph.add_trace(go.Scatter(x=n, y=timeJacobi,mode='lines',name='Jacobi'))
timeGraph.add_trace(go.Scatter(x=n, y=timeSeidel,mode='lines',name='Seidel'))
timeGraph.show()


itGraph = timeGraph = go.Figure()
itGraph.update_layout(
    title="Gráfico de Iterações - Sistemas Densos",
    xaxis_title="Ordem da Matriz",
    yaxis_title="Tempo em ms",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)

itGraph.add_trace(go.Scatter(x=n, y=itGauss,mode='lines',name='Gauss'))
itGraph.add_trace(go.Scatter(x=n, y=itLu,mode='lines',name='Lu'))
itGraph.add_trace(go.Scatter(x=n, y=itJacobi,mode='lines',name='Jacobi'))
itGraph.add_trace(go.Scatter(x=n, y=itSeidel,mode='lines',name='Seidel'))
itGraph.show()

[0.9262561798095703, 3.255605697631836, 6.010770797729492, 11.614561080932617, 16.463518142700195, 22.75371551513672, 33.150672912597656, 44.17276382446289, 52.434444427490234, 65.72389602661133, 76.95770263671875, 102.61750221252441, 111.8767261505127, 137.58611679077148, 167.82426834106445, 189.87584114074707, 236.41490936279297, 267.3652172088623, 344.61045265197754, 267.075777053833, 313.52782249450684, 346.8351364135742, 372.5879192352295, 484.4989776611328, 502.8870105743408, 506.4890384674072, 545.335054397583, 549.1256713867188, 601.6454696655273, 622.4198341369629]
[1.4450550079345703, 10.956287384033203, 31.011104583740234, 76.24673843383789, 140.1512622833252, 251.42312049865723, 395.8406448364258, 602.0543575286865, 817.2280788421631, 1139.357566833496, 1506.5233707427979, 2142.148494720459, 2521.472930908203, 3319.7174072265625, 4016.944408416748, 4924.27659034729, 5699.175834655762, 6950.4554271698, 9529.84619140625, 9102.828979492188, 10666.546106338501, 12503.9098262786

#**Gráfico dos sistemas esparsos**

In [ ]:
n = [10]

for i in range(20, 301, 10):
  n.append(i)

timeGauss = []
timeLu = []
timeJacobi = []
timeSeidel = []

itGauss = []
itLu = []
itJacobi = []
itSeidel = []

for item in n:
  A0, b = criarMatrizEsparsa(item)
  A1 = A0.copy()
  A2 = A0.copy()
  A3 = A0.copy()

  #print(f'Solve: {solve(A0,b)}')

  timeStart = time.time()
  G = gauss(A0,b)
  timeEnd = time.time()
  timeGauss.append((timeEnd - timeStart)*1000)
  itGauss.append(G[1])

  timeStart = time.time()
  M = magalu(A1,b)
  timeEnd = time.time()
  timeLu.append((timeEnd - timeStart)*1000)
  itLu.append(M[1])

  timeStart = time.time()
  J = jacobi(A2,b, 25)
  timeEnd = time.time()
  timeJacobi.append((timeEnd - timeStart)*1000)
  itJacobi.append(J[1])

  timeStart = time.time()
  S = gauss_seidel(A3,b, 25)
  timeEnd = time.time()
  timeSeidel.append((timeEnd - timeStart)*1000)
  itSeidel.append(S[1])

print(timeGauss)
print(timeLu)
print(timeJacobi)
print(timeSeidel)
print(itGauss)
print(itLu)
print(itJacobi)
print(itSeidel)

timeGraph = go.Figure()
timeGraph.update_layout(
    title="Gráfico do Tempo de Execução - Sistemas Esparsos",
    xaxis_title="Ordem da Matriz",
    yaxis_title="Tempo em ms",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)

timeGraph.add_trace(go.Scatter(x=n, y=timeGauss,mode='lines',name='Gauss'))
timeGraph.add_trace(go.Scatter(x=n, y=timeLu,mode='lines',name='Lu'))
timeGraph.add_trace(go.Scatter(x=n, y=timeJacobi,mode='lines',name='Jacobi'))
timeGraph.add_trace(go.Scatter(x=n, y=timeSeidel,mode='lines',name='Seidel', line=dict(color="#ff0000")))
timeGraph.show()


itGraph = go.Figure()
itGraph.update_layout(
    title="Gráfico de Iterações - Sistemas Esparsos",
    xaxis_title="Ordem da Matriz",
    yaxis_title="Tempo em ms",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)

itGraph.add_trace(go.Scatter(x=n, y=itGauss,mode='lines',name='Gauss'))
itGraph.add_trace(go.Scatter(x=n, y=itLu,mode='lines',name='Lu'))
itGraph.add_trace(go.Scatter(x=n, y=itJacobi,mode='lines',name='Jacobi'))
itGraph.add_trace(go.Scatter(x=n, y=itSeidel,mode='lines',name='Seidel', line=dict(color="#ff0000")))
itGraph.show()

[0.5106925964355469, 2.1772384643554688, 5.736351013183594, 9.148359298706055, 13.84878158569336, 22.190570831298828, 33.51426124572754, 42.10352897644043, 55.42588233947754, 71.45833969116211, 80.18898963928223, 99.02358055114746, 109.33780670166016, 138.60511779785156, 181.68258666992188, 244.08769607543945, 264.7898197174072, 248.70753288269043, 282.0284366607666, 298.75755310058594, 314.5442008972168, 352.247953414917, 398.78249168395996, 424.3130683898926, 466.63784980773926, 521.9783782958984, 552.3843765258789, 604.5281887054443, 654.0873050689697, 718.0814743041992]
[1.3120174407958984, 10.754823684692383, 32.157182693481445, 72.01862335205078, 159.07931327819824, 270.99609375, 469.23184394836426, 667.6766872406006, 831.9511413574219, 1143.6686515808105, 1680.1214218139648, 1950.089454650879, 2611.6418838500977, 3152.109146118164, 4077.709436416626, 5589.167594909668, 6575.850248336792, 6691.050052642822, 7845.752716064453, 9015.28000831604, 10324.039220809937, 11994.1012859344